In [2]:
# TAG: Import Library
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle
from skimage.feature import hog
from collections import Counter
from mlxtend.data import loadlocal_mnist

In [3]:
# TAG: Load EMNIST Letters Dataset (2600 samples - 100 per class)
X_raw, y_raw = loadlocal_mnist(
    images_path=r'C:\Users\HP\Downloads\emnist_source_files\emnist-letters-train-images-idx3-ubyte',
    labels_path=r'C:\Users\HP\Downloads\emnist_source_files\emnist-letters-train-labels-idx1-ubyte'
)

X_raw = X_raw.reshape(-1, 28, 28)  # reshape menjadi (n, 28, 28)
y_raw = y_raw - 1  # ubah label 1-26 menjadi 0-25 (A-Z)

In [ ]:
# TAG: Ambil 100 sample per kelas
selected_X, selected_y = [], []
for label in np.unique(y_raw):
    idx = np.where(y_raw == label)[0][:100]
    selected_X.append(X_raw[idx])
    selected_y.append(y_raw[idx])
X = np.concatenate(selected_X)
y = np.concatenate(selected_y)

from collections import Counter
print("Jumlah sample per kelas:", Counter(y))

In [ ]:
# TAG: Visualisasi beberapa gambar
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X[i], cmap='gray')
    plt.title(f"Label: {chr(y[i]+65)}")
    plt.axis('off')
plt.suptitle("Contoh Gambar dari EMNIST")
plt.show()

In [ ]:
# TAG: Shuffle Dataset
X, y = shuffle(X, y, random_state=42)

In [ ]:
# TAG: Ekstraksi Fitur HOG dengan parameter dimodifikasi
def extract_hog_features(images):
    hog_features = []
    for img in images:
        features = hog(img,
                       orientations=9,
                       pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2),
                       block_norm='L2-Hys',
                       visualize=False)
        hog_features.append(features)
    return np.array(hog_features)

X_hog = extract_hog_features(X)
print("Ukuran fitur HOG:", X_hog.shape)

In [ ]:
# TAG: Simulasi Leave-One-Out Cross Validation (split 80/20)
split_idx = int(0.8 * len(X_hog))
X_train, X_test = X_hog[:split_idx], X_hog[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

In [ ]:
# TAG: Grid Search SVM
param_grid = {
    'C': [1, 10],
    'gamma': [0.001, 0.01],
    'kernel': ['rbf']
}

svm = SVC()
grid = GridSearchCV(svm, param_grid, cv=5)
grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)

In [ ]:
# TAG: Evaluasi Training
model = grid.best_estimator_
y_train_pred = model.predict(X_train)
print("--- Training Performance ---")
print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("Precision:", precision_score(y_train, y_train_pred, average='macro'))
print("Recall:", recall_score(y_train, y_train_pred, average='macro'))
print("F1 Score:", f1_score(y_train, y_train_pred, average='macro'))

In [ ]:
# TAG: Evaluasi Testing
y_test_pred = model.predict(X_test)
print("\n--- Testing Performance ---")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred, average='macro'))
print("Recall:", recall_score(y_test, y_test_pred, average='macro'))
print("F1 Score:", f1_score(y_test, y_test_pred, average='macro'))